In [4]:
import cv2
import numpy as np
from numpy.linalg import norm
import sys
import os
import json
#自定义类
from plate_recognition_utils.plate_recog import plate_recognition

In [10]:
img_path="./cars_image/2.jpg"

#检测常见车牌长宽比
wh_ratio_low=1.9
wh_ratio_high=5.5

Min_Area = 2000
kernel_size=3
blur=19

'''初始化实例'''
plate_recog=plate_recognition.initialize(img_path,wh_ratio_low,wh_ratio_high,Min_Area,kernel_size,blur)

'''
图像处理第二步
实现功能
1：对上步骤返回的掩码图像查找contours
2：筛选出contourArea大于2000的轮廓
'''
mask_blue, mask_green, mask_yellow=plate_recog.plate_inRange()

In [11]:
'''绘出掩码图片'''
plate_recognition.draw(img_path,mask_blue, mask_green, mask_yellow)

In [12]:
'''
图像处理第二步
实现功能
1：对上步骤返回的掩码图像查找contours
2：筛选出contourArea大于2000的轮廓
'''
contours_b, contours_g, contours_y=plate_recog.findContours(mask_blue, mask_green, mask_yellow)

'''
图像处理第三步
实现功能
1：调用子函数plate_box_selection去除不符合长宽比，角度的contours
'''
contours,color_mark=plate_recog.plate_boxes_selection(contours_b,contours_g,contours_y)

'''
图像处理第四步
实现功能
1：调用子函数crop_rect对每个contour进行旋转操作
'''
img_crops=plate_recog.crop_recycle(contours)

'''
图像处理第五步：
实现代码
1：将黄色图像做颜色反向处理,以便于后期做图像二值化处理
'''
img_crops=plate_recog.reverse_yellow_plate(img_crops,color_mark)

'''
图像处理第六步：
实现功能
根据颜色比例筛选不合格contour
'''
crops,color_mark,contours=plate_recog.crops_filter_color(img_crops,color_mark,contours)

'''
图片处理第七步：
实现功能
通过子函数crop_cut_ylim check_black_rim check_white_rim 对车牌边框进行切除
'''
crops,contours=plate_recog.cut_rim(crops,color_mark,contours)

'''
图像处理第八步：
实现功能
车牌的二值化图每个字符之间是有间隔的，根据波谷去除不合格数据
'''
crops,color_mark,contours=plate_recog.crops_filter_by_vally(crops,color_mark,contours)

'''
图像处理第九步：
实现功能：
容易受到树边框和蓝色路牌的影响，由于树和蓝色路牌一般在图片的上半部分，我们这里去掉图像上1/3的框
注：经验主义，不赞同的同学可以pass这一步
'''
crops,color_mark,contours=plate_recog.crops_filter_by_common(crops,color_mark,contours)

'''
图像处理最后一步：
实现功能：
将剩下的框中面积最大的一个框作为车牌
'''
plate,color_mark=plate_recog.crops_filter_size(crops,color_mark)

In [13]:
'''
目前github上绝大部分车牌抠取采用的是
第一步：threshold,inrange,findcoutours根据车牌白边框找到contours
第二步：再根据找到的框筛选颜色，做一层层处理，
但是经实践，我发现这种方法在第一步正确定位框效率就较低，画出的框边界不贴合。且由于找到的框太多，后期筛选也相对困难

我这里采用hsv,inrange创建掩码，根据颜色分别过滤筛选蓝，黄，绿三种颜色车牌，经实践，效率高，且contours边界贴合
但是存在一个问题，对蓝车蓝牌，黄车黄牌不敏感

于是，这里我对两种定位思路进行融合，在上述代码没有找到车牌的情况下，
将采用以下方法进行车牌再次定位，以查缺补漏
'''

if color_mark!=0:
    cv2.imshow("plate", plate)
    cv2.waitKey(0)
    cv2.destroyAllWindows()  
else:
    contours=plate_recog.find_contours_by_Canny()
    contours=plate_recog.plate_box_selection(contours)
    img_crops=plate_recog.crop_recycle(contours)
    color_mark=[1]*len(img_crops)
    crops,color_mark,contours=plate_recog.crops_filter_color(img_crops,color_mark,contours)
    crops,contours=plate_recog.cut_rim(crops,color_mark,contours)
    crops,color_mark,contours=plate_recog.crops_filter_by_vally(crops,color_mark,contours)
    crops,color_mark,contours=plate_recog.crops_filter_by_common(crops,color_mark,contours)
    plate,color_mark=plate_recog.crops_filter_size(crops,color_mark)
    if color_mark!=0:
        cv2.imshow("plate", plate)
        cv2.waitKey(0)
        cv2.destroyAllWindows() 
    else:
        print('no plate found')

In [10]:
plate_path="./plates_image/9_%d.jpg" % color_mark

cv2.imwrite(plate_path,plate)

True